## Setting environment

In [1]:
!curl -O https://raw.githubusercontent.com/deepjavalibrary/d2l-java/master/tools/colab_build.sh && bash colab_build.sh
!java --list-modules | grep "jdk.jshell"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   520  100   520    0     0   4262      0 --:--:-- --:--:-- --:--:--  4262
Update environment...
Install Java...
Install Jupyter java kernel...
jdk.jshell@11.0.14.1


In [2]:
!git clone https://github.com/allenai/s2orc-doc2json.git
import os
os.chdir('/content/s2orc-doc2json')
!pip install -r requirements.txt

Cloning into 's2orc-doc2json'...
remote: Enumerating objects: 480, done.
remote: Counting objects: 100% (279/279), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 480 (delta 166), reused 181 (delta 88), pack-reused 201
Receiving objects: 100% (480/480), 9.40 MiB | 29.98 MiB/s, done.
Resolving deltas: 100% (255/255), done.
     |████████████████████████████████| 94 kB 1.4 MB/s 
     |████████████████████████████████| 128 kB 33.2 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 91 kB 6.1 MB/s 
     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 70 kB 4.8 MB/s 
     |████████████████████████████████| 5.7 MB 25.8 MB/s 
     |████████████████████████████████| 58 kB 4.3 MB/s 
     |████████████████████████████████| 547 kB 47.8 MB/s 
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstal

### interupt cell when output is stuck in *EXECUTING 87%*

In [3]:
!bash scripts/setup_grobid.sh

<===========--> 87% EXECUTING  39s]<===========--> 87% EXECUTING  40s]<===========--> 87% EXECUTING  41s]

In [4]:
import subprocess
subprocess.Popen(["bash", "scripts/run_grobid.sh"])

## Pdf2Json

In [5]:
import json
import argparse
import time
from bs4 import BeautifulSoup
from typing import Optional, Dict

from doc2json.grobid2json.grobid.grobid_client import GrobidClient
from doc2json.grobid2json.tei_to_json import convert_tei_xml_file_to_s2orc_json, convert_tei_xml_soup_to_s2orc_json

BASE_TEMP_DIR = 'temp'
BASE_OUTPUT_DIR = 'output'
BASE_LOG_DIR = 'log'

def process_pdf_stream(input_file: str, sha: str, input_stream: bytes, grobid_config: Optional[Dict] = None) -> Dict:
    """
    Process PDF stream
    :param input_file:
    :param sha:
    :param input_stream:
    :return:
    """
    # process PDF through Grobid -> TEI.XML
    client = GrobidClient(grobid_config)
    tei_text = client.process_pdf_stream(input_file, input_stream, 'temp', "processFulltextDocument")

    # make soup
    soup = BeautifulSoup(tei_text, "xml")

    # get paper
    paper = convert_tei_xml_soup_to_s2orc_json(soup, input_file, sha)

    return paper.release_json('pdf')


def process_pdf_file(
        input_file: str,
        temp_dir: str = BASE_TEMP_DIR,
        output_dir: str = BASE_OUTPUT_DIR,
        grobid_config: Optional[Dict] = None
) -> str:
    """
    Process a PDF file and get JSON representation
    :param input_file:
    :param temp_dir:
    :param output_dir:
    :return:
    """
    os.makedirs(temp_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # get paper id as the name of the file
    paper_id = '.'.join(input_file.split('/')[-1].split('.')[:-1])
    tei_file = os.path.join(temp_dir, f'{paper_id}.tei.xml')
    output_file = os.path.join(output_dir, f'{paper_id}.json')

    # check if input file exists and output file doesn't
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"{input_file} doesn't exist")
    # if os.path.exists(output_file):
    #     print(f'{output_file} already exists!')

    # process PDF through Grobid -> TEI.XML
    client = GrobidClient(grobid_config)
    # TODO: compute PDF hash
    # TODO: add grobid version number to output
    client.process_pdf(input_file, temp_dir, "processFulltextDocument")

    # process TEI.XML -> JSON
    assert os.path.exists(tei_file)
    paper = convert_tei_xml_file_to_s2orc_json(tei_file)

    # write to file
    with open(output_file, 'w') as outf:
        json.dump(paper.release_json(), outf, indent=4, sort_keys=False)

    return paper,output_file
os.makedirs(BASE_TEMP_DIR, exist_ok=True)
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)

In [ ]:
input_path = '/content/s2orc-doc2json/tests/pdf/N18-3011.pdf'
start_time = time.time()
paper,tei_file = process_pdf_file(input_path, BASE_TEMP_DIR, BASE_OUTPUT_DIR)
runtime = round(time.time() - start_time, 3)
print("runtime: %s seconds " % (runtime))
## Output file in /content/s2orc-doc2json/output/xx.json

/content/s2orc-doc2json/tests/pdf/N18-3011.pdf


ConnectionError: ignored

In [6]:
parent_path = '/content/'
for file in os.listdir(parent_path):
  if file.endswith('.pdf'):
    pdf_path = os.path.join(parent_path, file)
    paper,tei_file = process_pdf_file(pdf_path, BASE_TEMP_DIR, BASE_OUTPUT_DIR)

/content/648.pdf
/content/SlowFast.pdf
/content/Timesformer.pdf
/content/Video Transformer Network.pdf
/content/ViT.pdf
/content/I3D.pdf
/content/MAE.pdf
/content/566.pdf
/content/ViViT.pdf
/content/Deep residual learning for image recognition.pdf
/content/739.pdf
/content/518.pdf
/content/556.pdf
Processing failed with error 500


AssertionError: ignored

In [ ]:
! cp -r /content/s2orc-doc2json/output/ /conten

In [ ]:
!zip output_json.zip /content/s2orc-doc2json/output/*

  adding: content/s2orc-doc2json/output/518.json (deflated 86%)
  adding: content/s2orc-doc2json/output/556.json (deflated 84%)
  adding: content/s2orc-doc2json/output/566.json (deflated 83%)
  adding: content/s2orc-doc2json/output/648.json (deflated 86%)
  adding: content/s2orc-doc2json/output/687.json (deflated 83%)
  adding: content/s2orc-doc2json/output/697.json (deflated 84%)
  adding: content/s2orc-doc2json/output/719.json (deflated 83%)
  adding: content/s2orc-doc2json/output/739.json (deflated 81%)


## tldr generation

### envirnoment setting

In [ ]:
!pip install datasets transformers rouge-score nltk
!apt install git-lfs
!pip install -U spacy
!python -m spacy download en_core_web_sm
import transformers
print(transformers.__version__)

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 4.0 MB 8.6 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 136 kB 38.3 MB/s 
     |████████████████████████████████| 212 kB 62.1 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 127 kB 46.8 MB/s 
     |████████████████████████████████| 6.5 MB 38.4 MB/s 
     |████████████████████████████████| 895 kB 33.5 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
     |████████████████████████████████| 271 kB 59.1 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 56.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling 

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
sum_model = AutoModelForSeq2SeqLM.from_pretrained("HenryHXR/t5-base-finetuned-scitldr")
sum_tokenizer = AutoTokenizer.from_pretrained("HenryHXR/t5-base-finetuned-scitldr")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [ ]:
del sum_model

In [ ]:
def get_aic_from_json(paper_json):
  with open(paper_json,'r') as f: 
    paper = json.load(f)
  body_text = paper['pdf_parse']['body_text']
  text = paper['abstract']
  for index in range(len(body_text)):
    sec_name = body_text[index]['section'].lower()
    if sec_name.find('introduction')>=0 or sec_name.find('conclusion')>=0:
      text_toks = body_text[index].text.split(' ')
      if len(text_toks) > 60:
        print("sec:{}, text:{}".format(sec_name, body_text[index]['text']))
        text += body_text[index]['text']
  
  return text

  
def get_aic_from_obj(paper_json):
  
  body_text = paper_json.body_text
  text = [t.text for t in paper_json.abstract]
  text = ''.join(text)
  for index in range(len(body_text)):
    sec = body_text[index].section
    if sec:
      sec_name = sec[0][-1].lower()
      if sec_name.find('introduction')>=0 or sec_name.find('conclusion')>=0 or sec_name.find('discuss')>=0:
        
        text_toks = body_text[index].text.split(' ')
        if len(text_toks) > 60:
          print("sec:{}, text:{}".format(sec_name, body_text[index].text))
          text += body_text[index].text
  return text


import time
def generate_tldr(pdf_path,model,tokenizer):
  paper_tmp,tei_file = process_pdf_file(pdf_path,BASE_TEMP_DIR,BASE_OUTPUT_DIR)
  # time.sleep(5)
  article = get_aic_from_obj(paper_tmp)
  print(article)
  x = tokenizer("summarize: " + article, return_tensors="pt", max_length=1024, truncation=True)
  outputsd = model.generate(
      x["input_ids"], max_length=100, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
      )
  gen_tldr = tokenizer.decode(outputsd[0])
  print(gen_tldr)
  return gen_tldr.strip('<pad> </s>')

In [ ]:
input_path = '/content/s2orc-doc2json/tests/pdf/I3D.pdf'
tldr = generate_tldr(input_path,sum_model,sum_tokenizer)
print(tldr)

### named entity recognition

In [ ]:
from transformers import pipeline
ner_pipe = pipeline(task="ner",model='HenryHXR/scibert_scivocab_uncased-finetuned-ner')

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/417M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/700k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import spacy
nlp_sentence = spacy.load("en_core_web_sm")
label_list = ['O', 'B-Task', 'I-Task', 'B-Method', 'I-Method', 'B-OtherScientificTerm', 'I-OtherScientificTerm',
        'B-Generic', 'I-Generic', 'B-Material', 'I-Material', 'B-Metric', 'I-Metric']


In [ ]:

def scientific_term_recognition(ner_pipe, tldr, nlp_sentence, label_list):

  label_dict_list = {'Task':[],'Method':[],'OtherScientificTerm':[],
            'Generic':[],'Material':[],'Metric':[]}
  tldr_doc = nlp_sentence(tldr)
  for sen in tldr_doc.sents:
    # print('sentence text:',sen.text.strip())
    entities = ner_pipe(sen.text)
    # for entity in entities:
    #   print(entity)

    prev = ' '
    label_sen = [label_list[int(en['entity'][-1])] for en in entities]
    for en in entities:
      label_name = label_sen[en['index']-1]
      if label_name != 'O':

        label_name = label_name[2:]
        if prev != label_name:
          first_list = [en['word']]
          label_dict_list[label_name].append(first_list)
        else:
          label_dict_list[label_name][-1].append(en['word'])
      prev = label_name

  for key in label_dict_list:
    label_dict_list[key] = [" ".join(a).replace(' ##','').replace(' - ','-').replace(' ( ','(').replace(' )',')') for a in label_dict_list[key]]

  return label_dict_list

In [ ]:
input_path = '/content/ViT.pdf'
tldr = generate_tldr(input_path,sum_model,sum_tokenizer)
label_list_dict1 = scientific_term_recognition(ner_pipe, tldr, nlp_sentence, label_list)


sec:introduction, text:Self-attention-based architectures, in particular Transformers (Vaswani et al., 2017) , have become the model of choice in natural language processing (NLP). The dominant approach is to pre-train on a large text corpus and then fine-tune on a smaller task-specific dataset (Devlin et al., 2019) . Thanks to Transformers' computational efficiency and scalability, it has become possible to train models of unprecedented size, with over 100B parameters (Brown et al., 2020; Lepikhin et al., 2020) . With the models and datasets growing, there is still no sign of saturating performance.
sec:introduction, text:In computer vision, however, convolutional architectures remain dominant (LeCun et al., 1989; Krizhevsky et al., 2012; He et al., 2016) . Inspired by NLP successes, multiple works try combining CNN-like architectures with self-attention (Wang et al., 2018; Carion et al., 2020) , some replacing the convolutions entirely (Ramachandran et al., 2019; Wang et al., 2020a) 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<pad> We show that a pure Transformer applied directly to sequences of image patches can perform very well on image classification tasks compared to state-of-the-art convolutional networks. When pre-trained on large amounts of data and transferred to multiple mid-sized or small image recognition benchmarks</s>


In [ ]:
tldr

'We show that a pure Transformer applied directly to sequences of image patches can perform very well on image classification tasks compared to state-of-the-art convolutional networks. When pre-trained on large amounts of data and transferred to multiple mid-sized or small image recognition benchmark'

In [ ]:
out_str = ''
firstEnter = False
for key in label_list_dict1:
  if label_list_dict1[key]:
    label_one_key = ','.join(label_list_dict1[key])
    tmp_str = f'{key}: {label_one_key}'
    if firstEnter:
      out_str += '\n'
    else: firstEnter = True
    out_str+=tmp_str
print(out_str)

Task: image classification tasks,mid,sized or small image recognition benchmark
Method: pure transformer,convolutional networks
OtherScientificTerm: image patches
Generic: -
Material: -


In [ ]:
label_list_dict1

{'Generic': ['-'],
 'Material': ['-'],
 'Method': ['pure transformer', 'convolutional networks'],
 'Metric': [],
 'OtherScientificTerm': ['image patches'],
 'Task': ['image classification tasks',
  'mid',
  'sized or small image recognition benchmark']}